In [ ]:

import os
import time
import copy
import csv

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from PIL import Image

In [ ]:

# Set device (use GPU if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Data directories (modify these paths to match your dataset structure)
train_dir = '/kaggle/input/autistic-children-facial-data-set/train'
val_dir = '/kaggle/input/autistic-children-facial-data-set/test'
# Path for a single image for prediction
test_image_path = '/kaggle/input/autistic-children-facial-data-set/valid/autistic/01.jpg'


In [ ]:
# Data transforms for training and validation
data_transforms = {
    'train': transforms.Compose([
         transforms.Resize((224, 224)),
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
         transforms.Resize((224, 224)),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])
    ]),
}

In [ ]:
# Create datasets using ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(train_dir, data_transforms['train']),
    'val': datasets.ImageFolder(val_dir, data_transforms['val'])
}

# Create dataloaders for training and validation
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=4),
    'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=False, num_workers=4)
}

In [ ]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes  # list of class names
class_names

In [ ]:
# Load a pretrained EfficientNet (efficientnet_b0) and modify the classifier for binary classification
model = models.efficientnet_b0(pretrained=True)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)
model = model.to(device)

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Training parameters
num_epochs = 25
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

In [ ]:
# Lists to record loss and accuracy per epoch
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []

since = time.time()

# Training loop
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 30)
    
    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluation mode
            
        running_loss = 0.0
        running_corrects = 0
        
        # Iterate over data for the current phase
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass (and backpropagation during training)
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        
        print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        # Record history
        if phase == 'train':
            train_loss_history.append(epoch_loss)
            train_acc_history.append(epoch_acc.item())
        else:
            val_loss_history.append(epoch_loss)
            val_acc_history.append(epoch_acc.item())
            
            # Save the best model weights based on validation accuracy
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
    print()

time_elapsed = time.time() - since
print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed % 60:.0f}s')
print(f'Best validation accuracy: {best_acc:.4f}')

# Load best model weights and save to file
model.load_state_dict(best_model_wts)
torch.save(model.state_dict(), 'best_model_weights.pt')
print("Best model weights saved to 'best_model_weights.pt'")

# Save training history to a CSV file
with open('training_history.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc'])
    for epoch in range(num_epochs):
        writer.writerow([epoch+1, train_loss_history[epoch], train_acc_history[epoch],
                         val_loss_history[epoch], val_acc_history[epoch]])
print("Training history saved to 'training_history.csv'")

In [ ]:
# Function to predict the class of a single image
def predict_image(image_path, model, transform, class_names):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)  # Add batch dimension
    with torch.no_grad():
        outputs = model(image_tensor)
        _, preds = torch.max(outputs, 1)
    return class_names[preds[0]]

# Path for a single image for prediction
test_image_path = '/kaggle/input/autistic-children-facial-data-set/train/autistic/0004.jpg'

# Predict the class for a single image
prediction = predict_image(test_image_path, model, data_transforms['val'], class_names)
print(f'Prediction for the image "{test_image_path}" is: {prediction}')